In [1]:
import pandas as pd
import numpy as np
import fastparquet

In [3]:
from pandas.io import json
# Load data

csv1=pd.read_csv(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_0.csv")
csv2=pd.read_csv(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_1.csv")
csv3=pd.read_csv(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_2.csv")
json1=pd.read_json(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_3.json",lines=True)
json2=pd.read_json(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_4.json",lines=True)
json3=pd.read_json(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_7.json",lines=True)
json4=pd.read_json(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_9.json",lines=True)
json5=pd.read_json(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_10.json",lines=True)
parq1=pd.read_parquet(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_5.parquet")
parq2=pd.read_parquet(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_6.parquet")
parq3=pd.read_parquet(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_8.parquet")
parq4=pd.read_parquet(r"C:\Users\hatti\Downloads\fall2025_L(1)\ids_11.parquet")

list_of_files=[csv1,csv2,csv3,json1,json2,json3,json4,json5,parq1,parq2,parq3,parq4]


In [4]:
# Check for unique label
for files in list_of_files:
  print(files[' Label'].unique())

['BENIGN']
['BENIGN']
['BENIGN']
['BENIGN']
['BENIGN']
['DoS Hulk']
['DoS GoldenEye']
['DoS Slowhttptest' 'Heartbleed']
['DoS Hulk']
['DoS Hulk']
['DoS GoldenEye']
['DoS Hulk']


In [5]:
# Fix column names for all variables before concatenation:
for df in list_of_files:
    df.columns = df.columns.str.strip()

# Merge all the files into one
df_master = pd.concat(list_of_files, ignore_index=True)
df_master.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,300491,7,5,376,3508,201,0,53.714286,78.686539,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,37218,21,2,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,176,2,2,148,212,74,74,74.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,53,218,2,2,108,330,54,54,54.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,53,30753,2,2,70,126,35,35,35.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [6]:
df_master['Label'].unique()

array(['BENIGN', 'DoS Hulk', 'DoS GoldenEye', 'DoS Slowhttptest',
       'Heartbleed'], dtype=object)

In [7]:
# Check for null values and dtypes
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61128 entries, 0 to 61127
Data columns (total 79 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             61128 non-null  int64  
 1   Flow Duration                61128 non-null  int64  
 2   Total Fwd Packets            61128 non-null  int64  
 3   Total Backward Packets       61128 non-null  int64  
 4   Total Length of Fwd Packets  61128 non-null  int64  
 5   Total Length of Bwd Packets  61128 non-null  int64  
 6   Fwd Packet Length Max        61128 non-null  int64  
 7   Fwd Packet Length Min        61128 non-null  int64  
 8   Fwd Packet Length Mean       61128 non-null  float64
 9   Fwd Packet Length Std        61128 non-null  float64
 10  Bwd Packet Length Max        61128 non-null  int64  
 11  Bwd Packet Length Min        61128 non-null  int64  
 12  Bwd Packet Length Mean       61128 non-null  float64
 13  Bwd Packet Lengt

In [8]:
# Remove rows containing the label "HeartbLeed"
df_master = df_master[df_master['Label'] != 'Heartbleed']
print(df_master['Label'].unique())
df_master.shape[0]

['BENIGN' 'DoS Hulk' 'DoS GoldenEye' 'DoS Slowhttptest']


61117

In [9]:
# Replace Dos attacks with "ATTACK"
for labels in df_master['Label'].unique():
  if labels != 'BENIGN':
    df_master['Label'] = df_master['Label'].replace(labels, 'ATTACK')

print("labels after removing heartbleed and renaming Dos attacks are:",df_master['Label'].unique())

labels after removing heartbleed and renaming Dos attacks are: ['BENIGN' 'ATTACK']


In [10]:
# Check for changes in missing values after removing rows containing heartbleed
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61117 entries, 0 to 61127
Data columns (total 79 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             61117 non-null  int64  
 1   Flow Duration                61117 non-null  int64  
 2   Total Fwd Packets            61117 non-null  int64  
 3   Total Backward Packets       61117 non-null  int64  
 4   Total Length of Fwd Packets  61117 non-null  int64  
 5   Total Length of Bwd Packets  61117 non-null  int64  
 6   Fwd Packet Length Max        61117 non-null  int64  
 7   Fwd Packet Length Min        61117 non-null  int64  
 8   Fwd Packet Length Mean       61117 non-null  float64
 9   Fwd Packet Length Std        61117 non-null  float64
 10  Bwd Packet Length Max        61117 non-null  int64  
 11  Bwd Packet Length Min        61117 non-null  int64  
 12  Bwd Packet Length Mean       61117 non-null  float64
 13  Bwd Packet Length Std

In [11]:
# Remove INF values
numeric_df = df_master.select_dtypes(include=np.number)

# 2. Count rows containing infinite values
rows_with_inf = np.isinf(numeric_df).any(axis=1).sum()

print(f"Total number of flows (rows) containing at least one Inf value: {rows_with_inf}")

Total number of flows (rows) containing at least one Inf value: 83


In [12]:
# Create a copy of merged DataFrame
df_clean = df_master.copy()

# Replace all infinite values (Inf) with NaN
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop all rows that contain ANY NaN value (which now includes the Inf values)
df_clean.dropna(inplace=True)

# Verification
flows_removed = len(df_master) - len(df_clean)
print(f"Total rows removed due to infinite and null values: {flows_removed}")

Total rows removed due to infinite and null values: 116


In [13]:
# Convert 'BENIGN' to 0 and 'ATTACK' to 1 for scaling and training
df_clean['Label'] = df_clean['Label'].replace(['BENIGN', 'ATTACK'], [0, 1])

print("Labels converted:" ,df_clean['Label'].unique())

print("Rows and columns:",df_clean.shape)

Labels converted: [0 1]
Rows and columns: (61001, 79)


C:\Users\hatti\AppData\Local\Temp\ipykernel_23224\1472318422.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['Label'] = df_clean['Label'].replace(['BENIGN', 'ATTACK'], [0, 1])


In [14]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61001 entries, 0 to 61127
Data columns (total 79 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Destination Port             61001 non-null  int64  
 1   Flow Duration                61001 non-null  int64  
 2   Total Fwd Packets            61001 non-null  int64  
 3   Total Backward Packets       61001 non-null  int64  
 4   Total Length of Fwd Packets  61001 non-null  int64  
 5   Total Length of Bwd Packets  61001 non-null  int64  
 6   Fwd Packet Length Max        61001 non-null  int64  
 7   Fwd Packet Length Min        61001 non-null  int64  
 8   Fwd Packet Length Mean       61001 non-null  float64
 9   Fwd Packet Length Std        61001 non-null  float64
 10  Bwd Packet Length Max        61001 non-null  int64  
 11  Bwd Packet Length Min        61001 non-null  int64  
 12  Bwd Packet Length Mean       61001 non-null  float64
 13  Bwd Packet Length Std

In [15]:
df_clean.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
count,61001.00000,6.100100e+04,61001.000000,61001.000000,61001.000000,6.100100e+04,61001.000000,61001.000000,61001.000000,61001.000000,...,61001.000000,6.100100e+04,6.100100e+04,6.100100e+04,6.100100e+04,6.100100e+04,6.100100e+04,6.100100e+04,6.100100e+04,61001.000000
mean,759.28811,4.189280e+07,5.866887,4.246668,384.801331,7.268506e+03,261.028082,12.483845,61.797620,102.173976,...,29.677022,4.299848e+05,3.200237e+04,4.565406e+05,4.047415e+05,3.677495e+07,9.960781e+05,3.775972e+07,3.596751e+07,0.918083
std,5734.53197,4.263155e+07,70.995621,95.566899,1811.249897,2.205572e+05,314.824517,127.001542,144.872342,119.431075,...,5.513632,1.588729e+06,4.122231e+05,1.678795e+06,1.556942e+06,4.274586e+07,5.291552e+06,4.280229e+07,4.304179e+07,0.274240
min,0.00000,1.000000e+00,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,80.00000,1.745140e+05,2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,32.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000
50%,80.00000,1.197444e+07,6.000000,5.000000,337.000000,7.328000e+03,326.000000,0.000000,47.000000,123.458495,...,32.000000,5.710000e+02,0.000000e+00,5.710000e+02,4.850000e+02,6.901511e+06,0.000000e+00,6.901511e+06,6.866726e+06,1.000000
75%,80.00000,8.577019e+07,8.000000,6.000000,398.000000,1.159500e+04,374.000000,0.000000,68.571429,156.855256,...,32.000000,1.997000e+03,0.000000e+00,1.997000e+03,1.990000e+03,8.510000e+07,0.000000e+00,8.530000e+07,8.510000e+07,1.000000
max,63276.00000,1.199919e+08,17487.000000,23539.000000,278248.000000,5.440000e+07,20440.000000,1983.000000,4352.085714,5185.441640,...,60.000000,1.420000e+07,1.010000e+07,2.630000e+07,1.370000e+07,1.200000e+08,6.080000e+07,1.200000e+08,1.200000e+08,1.000000


In [16]:
# Check for duplicates
df_clean.columns.duplicated().any()


np.False_

In [17]:
# Prepare dataset for scaling
X = df_clean.drop('Label', axis=1).copy()
y = df_clean['Label'].copy()

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaling successful!")



Scaling successful!


In [19]:
print("Standardized mean: ",np.mean(X_scaled))
print("Standardized std: ",np.std(X_scaled))

Standardized mean:  -1.0226391106840128e-17
Standardized std:  0.9336995618478524


In [20]:
# Saving data for modeling

import joblib

joblib.dump(X_scaled, "X_scaled.pkl")
joblib.dump(y, "y.pkl")
joblib.dump(X.columns.tolist(), "feature_names.pkl")

['feature_names.pkl']